# Demo 2: Create Mosaic Dataset to Manage Raster Data

In [6]:
import os
import requests
import getpass
import json
import arcpy

## Create Mosaic Dataset in Geodatabase (File Geodatabase or Enterprise Geodatabase)

In [8]:
# Option 1: Create File Geodatabase
arcpy.env.overwriteOutput = 1
demofolder = r"\\valyria\Z__DevSummit2020\Demo\ManageRaster"
mdgdb = "mosaicdataset_1.gdb"
arcpy.management.CreateFileGDB(demofolder, mdgdb)

# Create mosaic dataset
demows = os.path.join(demofolder, mdgdb)
mdname = "ndfd_4days_1"
sr = "4326"
# sr = "4326" # This is the EPSG code for the projection GCS_WGS_1984

arcpy.management.CreateMosaicDataset(demows, mdname, sr)


# # Option 2: Create Enterprise Database connection
# connectname = "rdvmdb03.sde"
# arcpy.CreateDatabaseConnection_management(demofolder,
#                                           connectname,
#                                           "POSTGRESQL",
#                                           "rdvmdb03.esri.com",
#                                           "DATABASE_AUTH",
#                                           "jie",
#                                           "jie",
#                                           "SAVE_USERNAME",
#                                           "sde")

# # Create mosaic dataset
# demows = os.path.join(demofolder, connectname)
# mdname = "ndfd_14days"
# sr = "GCS_WGS_1984"
# # sr = "4326" # This is the EPSG code for the projection GCS_WGS_1984

# arcpy.CreateMosaicDataset_management(demows, mdname, sr)

mdpath = os.path.join(demows, mdname)

## Add Raster data to mosaic dataset

In [9]:
# NDFD temporature forecast for the next 4 days from 3/16/20
data = r"\\valyria\Z__DevSummit2020\Demo\ManageRaster\ndfd\ds.temp__031220.bin"
rastertype = "GRIB"
# Note: auxillary input option allows configuration of raster type settings
auxoption = "Variable 'T@HTGL'"

# Add data to mosaic dataset, can either add single dataset or the entire folder. 
arcpy.management.AddRastersToMosaicDataset(mdpath, rastertype, data, aux_inputs=auxoption, estimate_statistics="ESTIMATE_STATISTICS")

<Result '\\\\valyria\\Z__DevSummit2020\\Demo\\ManageRaster\\mosaicdataset.gdb\\ndfd_4days'>

## Customize Mosaic Dataset Table data

In [11]:
arcpy.management.BuildPyramidsandStatistics(mdpath, build_pyramids="NONE", calculate_statistics="CALCULATE_STATISTICS")

# Mosaic dataset has a table view, also support custom operation on Table
# e.g. Add a new field to the mosaic dataset table
newfield = "RegionAverage_F"
arcpy.management.AddField(mdpath, newfield, "DOUBLE")

# Read individual raster item from the table using cursor
rasfields = ["OBJECTID", "Raster"]
with arcpy.da.SearchCursor(mdpath, rasfields) as rcursor:
    with arcpy.da.UpdateCursor(mdpath, [newfield]) as cursor:
        for row in cursor: 
            rrow = rcursor.next()
            meantemp = float(arcpy.management.GetRasterProperties(rrow[1], "MEAN").getOutput(0))
            row[0] = 1.8 * (meantemp - 273) + 32
            cursor.updateRow(row)

## Share Mosaic Dataset as Image Service

In [12]:
# Create portal token if Server is federated with ArcGIS Enterprise
portal_sharing_url = "https://valyria.esri.com/portal/sharing/rest"

# Use server admin URL if Server is a standalone ArcGIS Server
server_url = "https://valyria.esri.com/server"
server_admin_url = "https://valyria.esri.com/server/admin"

username = input("Admin username: ")
password = getpass.getpass("Admin password: ")


# Generate publishing token (Federated Server)
gentokenurl = portal_sharing_url + "/generateToken"
params = {"f": "json", "username": username, "password": password, "expiration": "20", "client": "referer", "referer": server_url}

r = requests.post(gentokenurl, data=params, verify=False)
token = json.loads(r.text)["token"]


Admin username: admin
Admin password: ········


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [13]:
isdef = {
 "serviceName": "demo_ndfd",
 "type": "ImageServer",
 "description": "",
 "capabilities": "Image, Catalog, Metadata, Mensuration",
 "provider": "ArcObjects11",
 "clusterName": "default",
 "minInstancesPerNode": 0,
 "maxInstancesPerNode": 2,
 "instancesPerContainer": 1,
 "maxWaitTime": 60,
 "maxStartupTime": 300,
 "maxIdleTime": 1800,
 "maxUsageTime": 600,
 "loadBalancing": "ROUND_ROBIN",
 "isolationLevel": "HIGH",
 "configuredState": "STARTED",
 "recycleInterval": 24,
 "recycleStartTime": "00:00",
 "keepAliveInterval": -1,
 "private": False,
 "isDefault": False,
 "maxUploadFileSize": 0,
 "allowedUploadFileTypes": "",
 "properties": {
  "path": "\\\\valyria\\Z__DevSummit2020\\Demo\\ManageRaster\\mosaicdataset.gdb\\ndfd_4days",
  "cacheControlMaxAge": "43200",
  "colormapToRGB": "false",
  "defaultResamplingMethod": "1",
 },
 "extensions": [],
 "frameworkProperties": {},
 "datasets": []
}

In [14]:
server_createservice = server_admin_url + "/services/createService"
print(server_createservice)
print(token)

params = {"f": "json", "token": token, "service": json.dumps(isdef), "referer": server_url}
r = requests.post(server_createservice, data=params, verify=False)

print(r.text)

https://valyria.esri.com/server/admin/services/createService
w0bjrwngFQzektM62ld6jA0rfmOHvUGWmDikioEn580klfSJO49svEAqjieCcJE2p0k34OU1KBq0hE-ytxj9Nf7KrZ4q9uVp-dziSnoFoO4RozLnsk9DZDui_IwB2y0JfSVab-bgpRDZQ1EB_mIvfQ..


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{"status":"success"}
